In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

Text for the predictor is in the <code>document</code> variable.

In [33]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
predictorOutput = predictor.predict(document="What is your recommended color? The recommended color is red. Please use it.")
#predictorOutput = predictor.predict(document="Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.")

In [34]:
def generateClusters(predictorOutput, currSentence):
    #function borrowed from: https://github.com/ananyagup/AllenNLP-Coreference-Resolution-in-Python-Readable-clusters
    clusters = predictorOutput['clusters']
    document = predictorOutput['document']
    n = 0
    doc = {}
    for obj in document:
        doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
        n += 1

    clus_all = []
    cluster = []
    print(doc)
    for i in range(0, len(clusters)):
        one_cl = clusters[i]
        for count in range(0, len(one_cl)):
            obj = one_cl[count]
            for num in range((obj[0]), (obj[1]+1)):
                for n in doc:
                    if num == n:
                        cluster.append(doc[n])
        clus_all.append(cluster)
        cluster = []

    print(clus_all) #shows all coreferences
    return clus_all

In [36]:
# generateReferences takes in a context (1+ previous sentences)
# and a current sentence and replaces all broad references with  
# coreferences and the broad reference itself
# TODO: extend list of broad references
broad_refs = ["he", "she", "this", "that", "them", "it"]
def generateReferences(context, currSentence):
    perms = []
    clus_all = generateClusters(predictorOutput, currSentence)
    sentence = ""
    for cluster in clus_all:
        print("cluster: ", cluster)
        for word in cluster:
                if word in broad_refs:
                    modified = [currSentence.replace(word, newWord) for newWord in cluster if newWord not in broad_refs]
                    return list(set(modified))
                    
                    
# example: "Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers."
# print("modified sentence: ", generateReferences("Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.", "with whom he shared an enthusiasm for computers."))

# Bug found: output is modified sentence:  ['Please use your.', 'Please use color.', 'Please use The.', 'Please use recommended.']
# This is due to coreferences being > 1 word long... 
print("modified sentence: ", generateReferences("What is your recommended color? The recommended color is red. Please use it.", "Please use it."))
    

{0: 'What', 1: 'is', 2: 'your', 3: 'recommended', 4: 'color', 5: '?', 6: 'The', 7: 'recommended', 8: 'color', 9: 'is', 10: 'red', 11: '.', 12: 'Please', 13: 'use', 14: 'it', 15: '.'}
[['your', 'recommended', 'color', 'The', 'recommended', 'color', 'it']]
cluster:  ['your', 'recommended', 'color', 'The', 'recommended', 'color', 'it']
modified sentence:  ['Please use your.', 'Please use color.', 'Please use The.', 'Please use recommended.']
